In [128]:
import pandas as pd

def addValuesToTable(df, colName, table):
    for index,gene in enumerate(df.index):
        value = df.loc[gene, colName]
        table.loc[gene, colName] = transcriptomeValue
        
def getKuoValue(gene):
    start = table.loc[gene].loc["START"]
    strand = table.loc[gene].loc["STRAND"]
    ran = start - 1000
    if strand == '-':
        ran = start
        start = start + 1000
    
    # Reduce search space
    chrom = 'chr0' + gene[5]
    search = kuo.loc[chrom]
    return search[(search.loc[:,"START"] < start) & 
                  (search.loc[:,"END"] > ran)]
    
    

In [44]:
# Get general info for each gene: location, strand, start, end
genInfo = pd.read_csv("C_glabrata_CBS138_current_feature_mochi_LocationSet.txt",
                 sep='\t',header=(0))
table = genInfo.iloc[:, [1,2,3,4]]
table.index = table.iloc[:,3]
table = table.iloc[:,[0,1,2]]

# Add roetzer data column
roetzer = pd.read_csv("Roetzer_Table_MochiView_DatasetByGene.txt", sep='\t',
                     header = (0))
roetzerYap = roetzer.iloc[:,[0,5]]
roetzerYap.index = roetzer.iloc[:,0]
roetzerYap = roetzerYap.iloc[:,[1]]

addValuesToTable(roetzerYap, "Yap1 Stressed", table)
    

In [60]:
# Add Lelandais Transcriptome Data
lelandais = pd.read_csv('TranscriptomeDataMochi.txt', sep='\t',
                       header=(0))
lelandais.index = lelandais.iloc[:, 0]
lelandais = lelandais.iloc[:, [1,3]]

for i in lelandais.columns:
    addValuesToTable(lelandais, i, table)


                 START  ...  Mean log2 fold change Rep2
FEATURE_NAME            ...                            
CAGL0A00105g    2636.0  ...                         NaN
CAGL0A00110g    4809.0  ...                         NaN
CAGL0A00116g    4809.0  ...                         NaN
CAGL0A00132g   11739.0  ...                         NaN
CAGL0A00154g   15012.0  ...                         NaN
CAGL0A00165g   18876.0  ...                    0.517124
CAGL0A00187g   21757.0  ...                    0.517124
CAGL0A00209g   22260.0  ...                    0.517124
CAGL0A00231g   25241.0  ...                    0.517124
CAGL0A00253g   28586.0  ...                    0.517124
CAGL0A00275g   29069.0  ...                    0.517124
CAGL0A00297g   30775.0  ...                    0.517124
CAGL0A00319g   30978.0  ...                    0.517124
CAGL0A00341g   33056.0  ...                    0.517124
CAGL0A00363g   36349.0  ...                    0.517124
CAGL0A00385g   40371.0  ...                    0

In [69]:
table.columns = ['START', 'END', 'STRAND', 'Roetzer log2', 'Lelandais log2 Rep1',
       'Lelandais log2 Rep2']

print(table.head(5).loc[:, ["START", "END"]])

                START      END
FEATURE_NAME                  
CAGL0A00105g   2636.0   1608.0
CAGL0A00110g   4809.0   1608.0
CAGL0A00116g   4809.0   2671.0
CAGL0A00132g  11739.0  12689.0
CAGL0A00154g  15012.0  15233.0


In [130]:
# Read in Kuo data
kuo = pd.read_csv('Kuo_Data_MochiView_LocationDataset.txt', sep='\t')
kuo = kuo.iloc[:, [0,1,2,3,4]]
kuo.columns = ['START', 'END', 'REP1', 'REP2', 'REP3']


print(getKuoValue("CAGL0A00105g"))
print(table.loc["CAGL0A00105g"])
# print(ex.loc['chr0A'])

       START   END    REP1    REP2    REP3
chr0A   2721  2780  0.1067  0.1991  0.1102
chr0A   3103  3162  0.0611  0.0833  0.3266
chr0A   3281  3340  0.1057  0.1851  0.1564
START                  2636
END                    1608
STRAND                    -
Roetzer log2            NaN
Lelandais log2 Rep1     NaN
Lelandais log2 Rep2     NaN
Name: CAGL0A00105g, dtype: object
